In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from scipy.integrate import odeint
import scipy.constants as cons
import scipy.linalg as la
from qutip import *

from sympy import *
init_printing(use_unicode=True) # pretty printing
from Omega_p_func import Omega_p_func

# Symbolic Calculation of Optical Bloch Equations

In [ ]:
# Symbol definitions for density matrix components
rho00, rho01, rho0e, rho0r, rho10, rho11, rho1e, rho1r, rhoe0, rhoe1, rhoee, rhoer, rhor0, rhor1, rhore, rhorr = symbols('rho_00, rho_01, rho_0e, rho_0r, rho_10, rho_11, rho_1e, rho_1r, rho_e0, rho_e1, rho_ee, rho_er, rho_r0, rho_r1, rho_re, rho_rr')

# Symbol definitions for Hamiltonian components
delta, Delta, Delta_c, V = symbols('delta Delta Delta_c V')
Omega_0, Omega_1, Omega_c = symbols('Omega_0 Omega_1 Omega_c')
hbar = symbols('hbar')

# Symbol definitions for Lindbladian components
Gamma_e0, Gamma_e1, Gamma_r = symbols('Gamma_e0 Gamma_e1 Gamma_r')

rho = Matrix([[rho00, rho01, rho0e, rho0r], [rho10, rho11, rho1e, rho1r], [rhoe0, rhoe1, rhoee, rhoer], [rhor0, rhor1, rhore, rhorr]]) # Density Matrix
H_t = hbar * Matrix([[-delta, 0, Omega_0/2, 0], [0, 0, Omega_1/2, 0], [Omega_0/2, Omega_1/2, -Delta, Omega_c/2], [0, 0, Omega_c/2, -(Delta + Delta_c - V)]]) # Hamiltonian

# Atomic Transition Operators
p00 = Matrix([[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
p11 = Matrix([[0, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
pee = Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 0]])
prr = Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1]])

p01 = Matrix([[0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
p0e = Matrix([[0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
p0r = Matrix([[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])

p10 = Matrix([[0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
p1e = Matrix([[0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
p1r = Matrix([[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0]])

pe0 = Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0]])
pe1 = Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0]])
per = Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0]])

pr0 = Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0]])
pr1 = Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 1, 0, 0]])
pre = Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0]])

# Lindbladian Components
Le0 = Gamma_e0 * ((p0e * rho * pe0) -(pee*rho + rho*pee)/2)
Le1 = Gamma_e1 * ((p1e * rho * pe1) -(pee*rho + rho*pee)/2)
Lre = Gamma_r * ((per * rho * pre) -(prr*rho + rho*prr)/2)

# Lindbladian
L = Le0 + Le1 + Lre

# Lindbladian Master Equation
liouville = -(I/hbar) * ((H_t * rho) - (rho * H_t)) 
obe = liouville + L

cond = obe[0] + obe[5] + obe[10] + obe[15] 

if simplify(cond) == 0:
    print("The condition p00 + p11 + pee + prr = 0 holds")
else:
    print("Condition does not hold")

simplify(obe)
#simplify(H_t)
#simplify(rho)
#simplify(liouville)
#simplify(L)

# Steady State Solution of Optical Bloch Equations

In [ ]:
# -- EIT CNOT Gate Target Atom Steady State Solution

# Pulse duration and maximum probe rabi frequency
tau = 2                               # (\micro s)

Omega_p_max = 2 * np.pi * 0.68        # (MHz)

# f_e = 3
Omega_0_max = 2 * np.pi * 42          # (MHz)
Omega_1_max = 2 * np.pi * 14          # (MHz)
Omega_c = 2 * np.pi * 38              # (MHz)

# f_e = 4
#Omega_0_max = 2 * np.pi * 26         # (MHz)
#Omega_1_max = 2 * np.pi * 37         # (MHz)
#Omega_c = 2 * np.pi * 43             # (MHz)

# Coupling field detuning
d = 2 * np.pi * 0.28                  # (MHz)
D = 2 * np.pi * 1.34 * 10**(3)        # (MHz)
Dc = -D

# Potential
V = 2 * np.pi * 0                    # (MHz)

# Radiative decay rates
gamma_e0 = 2 * np.pi * 5.2            # (MHz)
gamma_e1 = 2 * np.pi * 5.2              # (MHz)
gamma_r = 2 * np.pi * 1 * 10**(-3)    # (MHz)

# Time range
t_range = np.arange(0, 2, 0.001)      # (\micro s)

probe_freq = np.empty(len(t_range))

sol = np.empty((16, len(t_range)), dtype='complex')
for t in range(len(t_range)):

    # Rabi frequencies for probe and coupling fields
    Omega_p = Omega_p_func(t_range[t], Omega_p_max, tau)
    Omega_0 = Omega_p_func(t_range[t], Omega_0_max, tau)
    Omega_1 = Omega_p_func(t_range[t], Omega_1_max, tau)
    #Omega_c = 6 * Omega_p

    #V = (40 * cons.hbar * (Omega_c**2))/(4 * D)
    
    #w0 = (Omega_p*1j)/2
    #w1 = (Omega_p*1j)/2
    
    w0 = (Omega_0*1j)/2
    w1 = (Omega_1*1j)/2
    wc = (Omega_c*1j)/2
    
    # Some variables for easy maintenance
    dG0e = -D*1j + d*1j - (gamma_e0 + gamma_e1)/2
    dG0r = d*1j - (D + Dc - V)*1j - gamma_r/2
    dG1e = -D*1j - (gamma_e0 + gamma_e1)/2
    dG1r = -(D + Dc - V)*1j - gamma_r/2
    dGe0 = D*1j - d*1j - (gamma_e0 + gamma_e1)/2
    dGe1 = D*1j - (gamma_e0 + gamma_e1)/2
    dGer = D*1j - (D + Dc - V)*1j - (gamma_e0 + gamma_e1 + gamma_r)/2
    dGr0 = -d*1j + (D + Dc - V)*1j - gamma_r/2
    dGr1 = (D + Dc - V)*1j - gamma_r/2
    dGre = -D*1j + (D + Dc - V)*1j - (gamma_e0 + gamma_e1 + gamma_r)/2
    
    drho = np.array([[0, 0, w0, 0, 0, 0, 0, 0, -w0, 0, gamma_e0, 0, 0, 0, 0, 0],                                # p00
                     [0, 0, 0, 0, 0, 0, w1, 0, 0, -w1, gamma_e1, 0, 0, 0, 0, 0],                                # p11
                     [0, 0, -w0, 0, 0, 0, -w1, 0, w0, w1, -(gamma_e0 + gamma_e1), wc, 0, 0, -wc, gamma_r],      # pee
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -wc, 0, 0, wc, -gamma_r],                                # prr
                     [0, d*1j, w1, 0, 0, 0, 0, 0, 0, -w0, 0, 0, 0, 0, 0, 0],                                    # p01
                     [w0, w1, dG0e, wc, 0, 0, 0, 0, 0, 0, -w0, 0, 0, 0, 0, 0],                                  # p0e
                     [0, 0, wc, dG0r, 0, 0, 0, 0, 0, 0, 0, -w0, 0, 0, 0, 0],                                    # p0r
                     [0, 0, 0, 0, w0, w1, dG1e, wc, 0, 0, -w1, 0, 0, 0, 0, 0],                                  # p1e
                     [0, 0, 0, 0, 0, 0, wc, dG1r, 0, 0, 0, -w1, 0, 0, 0, 0],                                    # p1r
                     [0, 0, 0, -w0, 0, 0, 0, -w1, 0, 0, wc, dGer, 0, 0, 0, -wc],                                # per
                     [0, 0, 0, 0, -d*1j, 0, w0, 0, -w1, 0, 0, 0, 0, 0, 0, 0],                                   # p10
                     [-w0, 0, 0, 0, -w1, 0, 0, 0, dGe0, 0, w0, 0, -wc, 0, 0, 0],                                # pe0
                     [0, -w0, 0, 0, 0, -w1, 0, 0, 0, dGe1, w1, 0, 0, -wc, 0, 0],                                # pe1
                     [0, 0, 0, 0, 0, 0, 0, 0, -wc, 0, 0, 0, dGr0, 0, w0, 0],                                    # pr0
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, -wc, 0, 0, 0, dGr1, w1, 0],                                    # pr1
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -wc, 0, w0, w1, dGre, wc],                                  # pre
                     [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]])
    
    b = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

    rho = la.lstsq(drho, b)[0]
    sol[:, t] = rho

    #probe_freq[t] = Omega_p

pop = sol[0] + sol[5] + sol[10] + sol[15] # total populations

# ---------- Plotting ----------
plt.style.use(['science', 'notebook', 'grid'])

# Populations
plt.plot(t_range, sol[0, :].real, label=r'$|0\rangle$', color='tab:blue')
plt.plot(t_range, sol[5, :].real, label=r'$|1\rangle$', color='tab:orange')
plt.plot(t_range, sol[10, :].real, label=r'$|e\rangle$', color='tab:green')
plt.plot(t_range, sol[15, :].real, label=r'$|r\rangle$', color='tab:red')

# Smooth pulse
probe_freq = Omega_p_func(t_range, Omega_p_max, tau)
plt.plot(t_range, (probe_freq/Omega_p_max), '--', label=r'$\Omega_p(t)/\Omega_p^{max}$', color='gray')

probe0_freq = Omega_p_func(t_range, Omega_0_max, tau)
#plt.plot(t_range, (probe0_freq/Omega_0_max), '--', label=r'$\Omega_0(t)/\Omega_0^{max}$', color='red')

probe1_freq = Omega_p_func(t_range, Omega_1_max, tau)
#plt.plot(t_range, (probe1_freq/Omega_1_max), '--', label=r'$\Omega_1(t)/\Omega_1^{max}$', color='orange')

# Total populations
#plt.plot(t_range, pop.real, label='Total Populations')

plt.xlim(0.005, tau - 0.005)
positions = np.arange(0.005, tau-0.005, 0.248)
labels = np.arange(0, tau+0.1, 0.25)
plt.xticks(positions, labels)

plt.title('Population vs. Time when $V(R)/2\pi = 0 MHz$')
plt.xlabel('Time [$\mu s$]')
plt.ylabel(r'Population ($\rho$)')

plt.legend(loc='upper right', bbox_to_anchor=(1.35, 0.75))
plt.savefig('Figures/EIT_CNOT_V0_pop.pdf')